In [20]:
import numpy as np

def generate_electrons(n_electrons, n_groups):
    # 生成每个群的随机三维坐标以及电荷，目前默认电子所以电荷数都是1.
    # 使用下面那一行注释内容就可以把电荷数扩充到1-10之间的随机数
    positions = np.random.randn(n_groups, n_electrons, 3)
    charges = -np.ones((n_groups, n_electrons), dtype=int)  # 每个电子带负电
    # charges = -np.random.randint(1, 11, size=(n_groups, n_electrons))  # 每个电子带负电1-10
    return positions, charges


def calculate_forces_and_potential(positions, charges):
    k_e = 1  # 库伦常数，用了原子单位制
    n_groups, n_electrons, _ = positions.shape
    forces = np.zeros_like(positions)
    potential_energy = np.zeros(n_groups, dtype=np.float32)

    for group in range(n_groups):
        for i in range(n_electrons):
            for j in range(i + 1, n_electrons):
                r_vec = positions[group, j] - positions[group, i]
                r_mag = np.linalg.norm(r_vec)
                if r_mag == 0:
                    continue  # 避免两个随机出来的坐标恰好完全相同
                r_hat = r_vec / r_mag

                force_magnitude = k_e * charges[group, i] * charges[group, j] / r_mag ** 2
                forces[group, i] -= force_magnitude * r_hat
                forces[group, j] += force_magnitude * r_hat
                potential_energy[group] += k_e * charges[group, i] * charges[group, j] / r_mag

    return forces, potential_energy



In [21]:
import torch

n_electrons = 5  # 每个电子群中的电子数量
n_groups = 10  # 生成10个电子群

# device = 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
positions, charges = generate_electrons(n_electrons, n_groups)
forces, potential_energy = calculate_forces_and_potential(positions, charges)
potential_energy = torch.from_numpy(potential_energy)
positions_tensor = torch.from_numpy(positions).to(device=device)
padding_tensor = torch.zeros((n_groups, 1, 3), device=device)
positions_tensor = torch.cat([padding_tensor, positions_tensor], dim=1)
positions_tensor.requires_grad = True

# 这里用来生成和处理数据

In [22]:
# 使用预定义的模型，为了简化，multi_hop_max_dist取0
# 如果使用的不是dp的服务器，请注释掉os.environ这两行
from transformers import GraphormerModel, AdamW, GraphormerConfig
import os

###
os.environ['HTTP_PROXY'] = 'http://ga.dp.tech:8118'
os.environ['HTTPS_PROXY'] = 'http://ga.dp.tech:8118'
###

model_name = "clefourrier/graphormer-base-pcqm4mv1"

backbone_model = GraphormerModel.from_pretrained(model_name)
config = GraphormerConfig.from_pretrained(model_name)
config.multi_hop_max_dist = 0

Some weights of GraphormerModel were not initialized from the model checkpoint at clefourrier/graphormer-base-pcqm4mv1 and are newly initialized: ['graph_encoder.emb_layer_norm.bias', 'graph_encoder.emb_layer_norm.weight', 'graph_encoder.graph_attn_bias.edge_dis_encoder.weight', 'graph_encoder.graph_attn_bias.edge_encoder.weight', 'graph_encoder.graph_attn_bias.graph_token_virtual_distance.weight', 'graph_encoder.graph_attn_bias.spatial_pos_encoder.weight', 'graph_encoder.graph_node_feature.atom_encoder.weight', 'graph_encoder.graph_node_feature.graph_token.weight', 'graph_encoder.graph_node_feature.in_degree_encoder.weight', 'graph_encoder.graph_node_feature.out_degree_encoder.weight', 'graph_encoder.layers.0.fc1.bias', 'graph_encoder.layers.0.fc1.weight', 'graph_encoder.layers.0.fc2.bias', 'graph_encoder.layers.0.fc2.weight', 'graph_encoder.layers.0.final_layer_norm.bias', 'graph_encoder.layers.0.final_layer_norm.weight', 'graph_encoder.layers.0.self_attn.k_proj.bias', 'graph_encoder

In [23]:
from transformers import GraphormerModel
import torch
import torch.nn as nn

# 定义新的注意力头，用于预测势能

class CustomGraphormerModel(nn.Module):
    def __init__(self, graphormer_model):
        super().__init__()
        self.graphormer = graphormer_model
        # self.force_head = nn.Linear(768, 3)  # 输出力的线性层，假设每个原子有三个力分量
        self.energy_head = nn.Linear(768, 1)  # 输出势能的线性层
    
    def convert_to_inputs(self, positions):
        n_groups, n_electrons, _ = positions.shape
        # 这是因为graphormer引入了一个虚节点，position[:, 0]都是占位符[0, 0, 0]，是没有任何意义的，所以force[:,1:]才有意义。真实的电子数是positions.shape[1]-1
        n_electrons -= 1
        device = 'cuda' if torch.cuda.is_available() else 'cpu' 
        # （服务器内存好像不太够，先用cpu了）
        # device = 'cpu'

        # 初始化Tensor
        # input_nodes = torch.ones((n_groups, n_electrons, 1), device=device, dtype=int)  # [num_graphs, num_nodes, 1]
        input_nodes = -torch.from_numpy(charges).unsqueeze(-1).to(device=device)
        input_edges = torch.zeros((n_groups, n_electrons, n_electrons, 1, 1), device=device, dtype=int) # [num_graphs, num_nodes, num_nodes, max_hop, num_edge_features]
        # attn_bias = torch.zeros((n_groups, n_electrons + 1, n_electrons + 1), device=device)  
        in_degree = torch.ones((n_groups, n_electrons), device=device, dtype=int) * (n_electrons - 1)
        out_degree = torch.ones((n_groups, n_electrons), device=device, dtype=int) * (n_electrons - 1)
        spatial_pos = torch.zeros((n_groups, n_electrons, n_electrons), device=device, dtype=int)  
        # spatial_pos = torch.arange(n_electrons).unsqueeze(0).repeat(n_groups, 1).to(device)
        attn_edge_type = torch.zeros((n_groups, n_electrons, n_electrons), device=device, dtype=int)

        # 计算边特征为距离的倒数
        for group in range(n_groups):
            for i in range(n_electrons):
                for j in range(n_electrons):
                    if i != j:
                        # distance = torch.norm(positions[group, i] - positions[group, j])
                        # attn_bias[group, i + 1, j + 1] = distance
                        input_edges[group, i, j, 0, 0] = 1
                        spatial_pos[group, i, j] = 1
        expanded_positions_i = positions.unsqueeze(2).repeat(1, 1, n_electrons + 1, 1)
        expanded_positions_j = positions.unsqueeze(1).repeat(1, n_electrons + 1, 1, 1)
        # print(expanded_positions_i.shape, expanded_positions_j.shape)

    # 计算所有配对的欧几里得距离
        attn_bias = torch.norm(expanded_positions_i - expanded_positions_j, dim=3)
        # print("positions_grad:", torch.autograd.grad(attn_bias[0][1][2], positions, create_graph=True)[0])

        graphormer_inputs = {
            'input_nodes': input_nodes,
            'input_edges': input_edges,
            'attn_bias': attn_bias,
            'in_degree': in_degree,
            'out_degree': out_degree,
            'spatial_pos': spatial_pos,
            'attn_edge_type': attn_edge_type
        }
        
        return graphormer_inputs

    def forward(self, positions):
        inputs = self.convert_to_inputs(positions=positions)
        outputs = self.graphormer(**inputs)
        pooled_output = outputs.last_hidden_state.mean(dim=1)  # 从Graphormer输出中提取池化表示
        # forces = self.force_head(pooled_output)  # 预测力  
        energy = self.energy_head(pooled_output)  # 预测势能
        # forces_1 = torch.zeros_like(positions)
        # print(torch.autograd.grad(energy[0], self.positions, create_graph=True, allow_unused=True))
        grad_outputs = torch.ones_like(energy)
        forces = -torch.autograd.grad(energy, positions, create_graph=True, retain_graph=True, grad_outputs=grad_outputs)[0]     
        # 用两种方法算了力，可以确定结果是对的。     
        # for i in range(energy.shape[0]): 
        #     # print(torch.autograd.grad(energy[i], self.positions, create_graph=True)[0][i])
        #     forces_1[i] = -torch.autograd.grad(energy[i], positions, retain_graph=True)[0][i] 
        # assert torch.allclose(forces, forces_1)
        return forces, energy
    


# positions_tensor = torch.from_numpy(positions).to(device=device)
# padding_tensor = torch.zeros((n_groups, 1, 3), device=device)
# positions_tensor = torch.cat([padding_tensor, positions_tensor], dim=1)
# positions_tensor.requires_grad = True
model = CustomGraphormerModel(backbone_model).to(device=device)

In [24]:
### use for debug

# positions_tensor.requires_grad = True

# expanded_positions_i = positions_tensor.unsqueeze(2).repeat(1, 1, n_electrons + 1, 1)
# expanded_positions_j = positions_tensor.unsqueeze(1).repeat(1, n_electrons + 1, 1, 1)
# attn_bias = torch.norm(expanded_positions_i - expanded_positions_j, dim=3)
# # attn_bias[0][1][2]
# print("positions_grad:", torch.autograd.grad(attn_bias[0][1][2], positions_tensor, create_graph=True)[0])
# pred_forces, pred_energy = model(positions_tensor)

In [10]:
# pred_energy
# potential_energy = torch.from_numpy(potential_energy)
# loss = loss_fn(pred_energy[0], potential_energy[0])
# loss.backward

tensor([[-0.5855],
        [-0.5877],
        [-0.5806],
        [-0.5816],
        [-0.5872],
        [-0.5863],
        [-0.5845],
        [-0.5875],
        [-0.5728],
        [-0.5861]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
learning_rate = 2e-5
parameters = list(model.parameters())
optimizer = AdamW(parameters, lr=learning_rate)
loss_fn = nn.L1Loss()

epochs = 3000
for epoch in range(epochs):
    epoch_loss = 0
    positions, charges = generate_electrons(n_electrons, n_groups)
    forces, potential_energy = calculate_forces_and_potential(positions, charges)
    potential_energy = torch.from_numpy(potential_energy).to(device=device, dtype=torch.float32)
    
    positions = torch.from_numpy(positions).to(device=device)
    padding_tensor = torch.zeros((n_groups, 1, 3), device=device)
    positions_tensor = torch.cat([padding_tensor, positions], dim=1).to(device=device)
    positions_tensor.requires_grad = True
    pred_forces, pred_energy = model(positions_tensor)
    pred_energy = pred_energy.squeeze()

    ### use energy to train ###
    # loss = loss_fn(pred_energy, potential_energy).to(device=device)
    ### use force to train ###
    loss = loss_fn(pred_forces[:, 1:], torch.from_numpy(forces).to(device=device))
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    
    if epoch % 100 == 0:
        print("-"*20, "Epoch", epoch, "-"*20)
        print("loss", loss)
        print("pred_force:", pred_forces[0,1:].cpu().detach().numpy())
        print("true force:", forces[0])
        # 当用导数作为损失的时候，原函数可能会相差一个常数。
        print("energy_diff:", pred_energy-potential_energy-min(pred_energy-potential_energy), "\n")
        # print("true energy:", potential_energy, "\n")
print("pred_energy:", pred_energy)
print("true energy:", potential_energy)


/opt/mamba/envs/hf/lib/python3.8/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


-------------------- Epoch 0 --------------------
loss tensor(0.6798, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
pred_force: [[ 0.6167466   0.35712984 -0.01489629]
 [-0.50477359 -0.15899799  0.19152649]
 [-0.66885715  0.60097575 -0.42632296]
 [ 0.15076018  0.39565822 -0.07277685]
 [-0.06851202 -0.72261752  0.11433603]]
true force: [[ 1.90920726 -0.56218909  0.44957415]
 [-0.61537038 -0.13940708  0.43540062]
 [-0.52754575  0.37265249 -0.45870219]
 [-0.97053222  1.07116783 -0.41202851]
 [ 0.20424109 -0.74222415 -0.01424406]]
energy_diff: tensor([2.7803, 1.0911, 1.3028, 2.6425, 2.1139, 2.5288, 1.5649, 1.3781, 0.0000,
        1.9802], device='cuda:0', grad_fn=<SubBackward0>) 

-------------------- Epoch 100 --------------------
loss tensor(0.6580, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
pred_force: [[-0.11296564 -0.33471499  0.16059682]
 [ 0.35784871  0.69445203  0.02294121]
 [-0.05639638 -0.31858402  0.25019831]
 [ 0.06134199 -0.45167513 -0.01688

In [17]:
potential_energy - pred_energy

tensor([7.6923, 7.6951, 7.7410, 7.9025, 7.7520, 7.7880, 8.3276, 7.6907, 7.6142,
        8.0990], grad_fn=<SubBackward0>)

In [25]:
pred_forces[0, 1:].detach().numpy(), forces[0]

(array([[ 0.14452395,  0.3603397 ,  0.08955894],
        [-0.09670379,  0.42073898, -0.99655067],
        [-0.68156128,  0.32000283,  0.55275471],
        [ 0.10994643, -0.68132027, -0.66032615],
        [ 0.45297355, -0.26133985,  0.47834905]]),
 array([[ 0.92717188,  0.27600466,  0.80216212],
        [-0.75420111,  0.59891876, -1.3950508 ],
        [-0.58388106,  0.07083495,  0.47482792],
        [-0.05501321, -0.75061304, -0.62454082],
        [ 0.46592351, -0.19514533,  0.74260158]]))